In [153]:
import numpy as np
from sklearn.cross_validation import KFold
arr = np.genfromtxt('data/chips.csv', delimiter=',')
xs, ys = arr[:,:2].copy(), np.int32(arr[:, 2:].ravel())
ys = 2 * ys - 1

In [154]:
def f1_score(predicted, true):
    tp = ((predicted == true) * (predicted == 1)).sum()
    tn = ((predicted == true) * (predicted == -1)).sum()
    fp = ((predicted != true) * (predicted == 1)).sum()
    fn = ((predicted != true) * (predicted == -1)).sum()
    p = tp*1.0/max(1, tp+fp)
    r = tp*1.0/max(1, tp+fn)
    return (2*p*r/max(1, p+r), tp, fn, fp, tn)

In [155]:
def shuffle(X, y):
    p = np.arange(0, X.shape[0])
    np.random.shuffle(p)
    return (X[p], y[p])
xs, ys = shuffle(xs, ys)

In [156]:
def split(X, y, part=10):
    p = X.shape[0] // part
    return X[p:], y[p:], X[:p], y[:p]

In [157]:
def kf_cross_validation(regressor, xs, ys, n_fold=10, times=1, **params):
    measure = lambda x, y: f1_score(x, y)[0]
    res = 0
    for _ in range(times):
        kf = KFold(len(xs), n_fold, True, 0)
        kf_sum = 0
        fold_num = 0
        for train_i, test_i in kf:
            regressor.fit(xs[train_i], ys[train_i], **params)
            predicted = regressor.predict(xs[test_i])
            meas = measure(predicted, ys[test_i])
            fold_num += 1
            kf_sum += meas
            print(ys[test_i])
            print("Time {}, fold {} done, measure = {}".format(_, fold_num, meas))
        res += kf_sum/n_fold
    return res/times

In [158]:
class SVM_SGD:
    def __init__(self, phi=None):
        self.phi = phi
        
    def fit(self, X, Y, C, lr=0.01, eps=1e-5, iters=1000):#K(x1, x2)=<phi(x1), phi(x2)>
        if self.phi is not None:
            X = np.apply_along_axis(self.phi, 1, X)
        teta = 1.0 / (2 * C)
        n = X.shape[0]
        dim = X.shape[1]

        last_obj= -np.inf
        t = lr
        lr = 1.

        #init value
        w = np.zeros(dim)
        w0 = 0
        for it in range(iters):
            for x,y in zip(X, Y):
                margin = y * (np.dot(x, w) - w0)
                sl = np.maximum(1 - margin, 0)
                w = w - lr * (-sl * y * x + 2 * teta * w)
                w0 = w0 - lr * (sl * y)
            margin = (np.dot(X, w) - w0) * Y
            cur_obj = np.maximum(1 - margin, 0).sum() + teta * np.dot(w, w)
            if abs(last_obj - cur_obj) < eps:
                break
            last_obj = cur_obj

            lr = lr*(1+lr*t*it)**-1
        self.w, self.w0 = w, w0
        #print("w = {}, w0 = {}".format(w, w0))
    
    def predict(self, x):
        if self.phi is not None:
            x = np.apply_along_axis(self.phi, 1, x)
        return np.ndarray.astype(np.sign(np.dot(x, self.w) - self.w0), np.int32)

In [159]:
def print_f1Nmatrix(t):
    print("confusion matrix")
    print(t[1], t[2])
    print(t[3], t[4])
    print("f1_score = ", t[0])

In [160]:
print("No kernel, C = 100")
svm = SVM_SGD()
svm.fit(xs, ys, 100)
pr = svm.predict(xs)
print_f1Nmatrix(f1_score(pr, ys))

No kernel, C = 100
confusion matrix
30 28
25 35
f1_score =  0.530973451327


In [161]:
print("phi (x, y, x**2+y**2)")
svm = SVM_SGD(lambda x: np.array([x[0], x[1], x[0]**2+x[1]**2]))
X_train, y_train, X_test, y_test = split(xs, ys, 10)
for C in [1, 10, 100, 1000, 10000]:
    print("C =", C)
    print("f1_score =", kf_cross_validation(svm, xs, ys, C=C))
    print("============================================")
    
print("Full dataset")
svm.fit(xs, ys, 100)
pr = svm.predict(xs)
print_f1Nmatrix(f1_score(pr, ys))
print("============================================")

phi (x, y, x**2+y**2)
C = 1
[-1  1 -1  1  1  1  1  1  1 -1 -1  1]
Time 0, fold 1 done, measure = 0.8
[-1  1  1 -1 -1 -1  1  1  1  1 -1  1]
Time 0, fold 2 done, measure = 0.7368421052631579
[ 1 -1 -1 -1 -1 -1  1 -1 -1  1  1  1]
Time 0, fold 3 done, measure = 0.5882352941176471
[ 1  1  1 -1 -1  1  1  1 -1  1  1 -1]
Time 0, fold 4 done, measure = 0.8
[-1  1  1  1  1 -1  1 -1 -1  1 -1  1]
Time 0, fold 5 done, measure = 0.7368421052631579
[-1  1  1 -1 -1 -1 -1 -1 -1 -1  1 -1]
Time 0, fold 6 done, measure = 0.4
[-1  1  1  1 -1 -1 -1 -1  1  1 -1  1]
Time 0, fold 7 done, measure = 0.6666666666666666
[-1 -1 -1 -1  1  1  1  1 -1 -1 -1  1]
Time 0, fold 8 done, measure = 0.5882352941176471
[ 1 -1 -1  1  1  1  1 -1 -1 -1 -1]
Time 0, fold 9 done, measure = 0.625
[ 1  1 -1  1 -1 -1 -1  1 -1 -1 -1]
Time 0, fold 10 done, measure = 0.5333333333333333
f1_score = 0.647515479876
C = 10
[-1  1 -1  1  1  1  1  1  1 -1 -1  1]
Time 0, fold 1 done, measure = 0.6153846153846154
[-1  1  1 -1 -1 -1  1  1  1  1 -1 